# Tclean mpi examples

This  notebook contains two examples of running CASA's tclean task in parallel:
* [Example 1](#example_1) - notebook server and MPI processes on the same single node
* [Example 2](#example_2) - notebook server and ip controller on one node, MPI processes on one or more other nodes
    

## Initial setup
Here are the steps I followed to setup and start this notebook server:

1\. begin with a monolithic CASA distribution in the current directory, e.g. casa-6.4.3-27

2\. create a virtual environment based on CASA's distributed version of Python

`casa-6.4.3-27/bin/python3 -m venv --system-site-packages venv`

3\. start the virtual environment 

`source venv/bin/activate`

4\. install additional packages

`pip install jupyterlab ipyparallel`

5\. start jupyter lab

`PATH=casa-6.4.3-27/lib/mpi/bin:$PATH JUPYTER_CONFIG_DIR="." IPYTHONDIR="." jupyter lab --no-browser --ip 0.0.0.0`

Some notes:  The --system-site-packages flag allows the virtual environment to link to packages installed in the parent environment, i.e. those included in the CASA distribution. Modifying the PATH allows mpiexec, etc. from the CASA distribution to be found. Modifying the IPython and Jupyter config directory overrides any user-specific settings in $HOME (if any). Starting Jupyter with --ip 0.0.0.0 allows connections from other IP addresses.


### Alternate setup

The following procedure is another way to combine CASA with Jupyter and is more consistent with the documentation for using modular CASA. Both tclean examples will also work with this method without any code changes. There are a couple caveats with this method: you may need to install the casadata package or create a config.py with paths to the data folders, and you will need to install or locate openmpi (I am unsure about other flavors like mpich). At NRAO the version that worked best for me was in /opt/casa/03/bin.  Avoid versions in /usr/lib64 as they may be inconsent across RHEL7 machines.   

1\. create a virtual environment based on python 3.6 or 3.8. At NRAO these are in /opt/local/bin.

`python3 -m venv venv`

2\. activate the environment

`source venv/bin/activate`

3\. upgrade the version of pip 

`pip install --upgrade pip`

4\. install additional packages, providing the path to mpicc [as discussed here](https://open-bitbucket.nrao.edu/projects/CASA/repos/casampi/browse/README.md) 

`MPICC=/opt/casa/03/bin/mpicc pip install casatasks casampi jupyterlab ipyparallel`

5\. start jupyter lab

`PATH=/opt/casa/03/bin:$PATH JUPYTER_CONFIG_DIR="." IPYTHONDIR="." jupyter lab --no-browser --ip 0.0.0.0`

## <a name="example_1"></a>Example 1
This is a simple example that starts 4 MPI processes on the local machine.  All processes need to then import start_mpi. The rank 0 process becomes the 'command client' and returns so that it can receive new commands while the other processes go into a serve() loop. This would typically happen automatically if the MPI processes imported either casatasks or casashell, but here we only import casatasks on the notebook kernel so we need to run this manually on the other processes. 

Then we issue the tclean command to the rank 0 process and it distributes work to the servers running the loop. Because of the 'with' statement, the controller and engines are automatically stopped when tclean is complete.

In [1]:
import ipyparallel as ipp
from casatasks import tclean

with ipp.Cluster(engines="mpi", n=4) as rc:
    rc[:].execute('import casampi.private.start_mpi')
    rc[0].apply_sync(tclean, vis='0420+417.ms', imagename='mpi_tclean_example1', niter=100, cell='1arcsec', parallel=True) 


Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.48s/engine]
Stopping engine(s): 1646610038
engine set stopped 1646610038: {'exit_code': 1, 'pid': 30849, 'identifier': 'ipengine-1646610037-togx-1646610038-30777'}
Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 30814, 'identifier': 'ipcontroller-1646610037-togx-30777'}


## <a name="example_2"></a>Example 2
This example goes through the individual steps to start the controller and sync up the processes. This provides greater flexibility and would be better suited for an environment where a user has multiple clusters or profiles, and/or where a user wants to start MPI processes on other nodes.

#### 2.1 define the cluster and start the controller

Note: controller_ip="*" tells the controller to accept connections from processes on other ip addresses.  Also, profile and cluster_id can be arbitrary names used to distinguish this notebook's resources from other clusters the user might be running. 

In [2]:
import ipyparallel as ipp

cluster = ipp.Cluster(engines="mpi", controller_ip="*", profile='default', cluster_id='1234')
await cluster.start_controller()
cluster

<Cluster(cluster_id='1234', profile='default', controller=<running>)>

#### 2.2 start and connect the engines

To demonstrate multi-node MPI I created a hostfile listing each node and the number of MPI processes per node that I wanted to start.  At NRAO this required that I have an interactive reservation on each listed node and that I setup ssh keys.  I would like to get this working through torque or slurm in a future example. 
Note: The absolute path to mpiexec seems to be required below.

In [3]:
with open('hosts.txt','w') as out1:
    out1.writelines([
        'nmpost035.aoc.nrao.edu slots=2 \n',
        'nmpost036.aoc.nrao.edu slots=3 \n'
    ])
    
from subprocess import Popen, PIPE    
p = Popen("<full path to>mpiexec -x PATH -x IPYTHONDIR --hostfile hosts.txt \
          ipengine --file=profile_default/security/ipcontroller-1234-engine.json".split(), stdout=PIPE, stderr=PIPE)

rc =  await cluster.connect_client()
rc.wait_for_engines(5)

100%|███████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.20s/engine]


#### 2.3 map engine ID to MPI rank
In Example 1 the engine ID and MPI rank always seemed to be equal so this step was not required, but here the engines can connect to the controller in an arbitrary order.  This code will fetch the MPI rank of each engine and use that to determine which engine has MPI rank 0.

In [4]:
def get_mpi_rank():
    from mpi4py import MPI
    return MPI.COMM_WORLD.Get_rank()
    
rank = rc[:].apply_async(get_mpi_rank).get()
rank0 = rank.index(0)
print(f"engine ID {rank0} has MPI rank 0")

engine ID 1 has MPI rank 0


#### 2.4 start mpicasa
Import casampi on all the processes as in Example 1. The rank 0 process will become the CASA MPI command client, and the other processes will become CASA server processes. 

In [5]:
rc[:].execute('import casampi.private.start_mpi')

<AsyncResult(execute): pending>

#### 2.5 run tclean
Submit the tclean call to the rank 0 process and wait for it to complete.

In [6]:
from casatasks import tclean

rc[rank0].apply_sync(tclean, vis='0420+417.ms', imagename='mpi_tclean_example2', niter=100, cell='1arcsec', parallel=True) 

{}

#### 2.6 stop the engines and the cluster

In [7]:
p.terminate()
await cluster.stop_cluster()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 31265, 'identifier': 'ipcontroller-1234-30777'}
